In [12]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from implementations import *
from helper import *
from feature_analysis import *
from expansions import polynomial_expansion

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from proj1_helpers import *

DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
DATA_TEST_PATH = '../data/test.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

i_PRI = 22
y_jet0  = y[tX[:, i_PRI]==0]
tx_jet0 = tX[tX[:, i_PRI]==0]

y_jet1  = y[ tX[:, i_PRI] == 1]
tx_jet1 = tX[tX[:, i_PRI] == 1]

y_jet2  = y[ tX[:, i_PRI] > 1]
tx_jet2 = tX[tX[:, i_PRI] > 1]
#----------------------------------
#Then it can be executed like this
tx_0_filtered = np.delete(tx_jet0, [4,5,6,12,22,23,24,25,26,27,28], axis=1)
tx_1_filtered = np.delete(tx_jet1, [4,5,6,12,22,26,27,28], axis=1)
tx_2_filtered = np.delete(tx_jet2, [22], axis=1)

tx_0_filtered = fix_nan_values(fix_missing_values(tx_0_filtered))
tx_1_filtered = fix_nan_values(fix_missing_values(tx_1_filtered))
tx_2_filtered = fix_nan_values(fix_missing_values(tx_2_filtered))

tx_train_0 = featureExpand(tx_0_filtered, 0)
tx_train_1 = featureExpand(tx_1_filtered, 1)
tx_train_2 = featureExpand(tx_2_filtered, 2)

In [8]:
y_jet0 = y_jet0.reshape((len(y_jet0), 1))
y_jet1 = y_jet1.reshape((len(y_jet1), 1))
y_jet2 = y_jet2.reshape((len(y_jet2), 1))

In [9]:
from logistic_regression import *

In [23]:
def find_best_logistic_regression_model(y, tx, max_iter=10, gamma=0.01, k_fold=4) :
   # lambdas = np.logspace(0, -10, 11)
    #lambdas = np.logspace(0, -20, 4)
    lambdas = [100, 1, 1e-5, 0]
    degrees = np.arange(1, 5) 

    results = np.zeros((len(degrees), len(lambdas)))

    for i_degree, degree in enumerate(degrees) : 
        tx = polynomial_expansion(tx, degree)
        for i_lambda, lambda_  in enumerate(lambdas) : 

            results[i_degree, i_lambda] = cross_validate_logistic_regression(y, tx, max_iter, gamma, lambda_, k_fold)
            print("degree={d},\t lambda={l},\taccuracy={a}".format(d=degree, l=lambda_, a=results[i_degree, i_lambda]))

    i,j = np.unravel_index(np.argmax(results, axis=None), results.shape)

    return degrees[i], lambdas[j], results[i, j]

In [24]:
find_best_logistic_regression_model(y_jet0, tx_0_filtered)

degree=1,	 lambda=100,	accuracy=0.7448654816238289
degree=1,	 lambda=1,	accuracy=0.7448654816238289
degree=1,	 lambda=1e-05,	accuracy=0.7448654816238289
degree=1,	 lambda=0,	accuracy=0.7448654816238289


KeyboardInterrupt: 

In [ ]:
from least_squares import cross_validate_least_squares 

def find_best_least_squares_model(y, tx, k_fold=4) :
   # lambdas = np.logspace(0, -10, 11)
    #lambdas = np.logspace(0, -20, 4)
    lambdas = [100, 1, 1e-5, 0]
    degrees = np.arange(1, 5) 

    results = np.zeros((len(degrees), len(lambdas)))

    for i_degree, degree in enumerate(degrees) : 
        tx = polynomial_expansion(tx, degree)
        for i_lambda, lambda_  in enumerate(lambdas) : 
            try : 
                results[i_degree, i_lambda] = cross_validate_least_squares(y, tx, lambda_, k_fold)
                print("degree={d},\t lambda={l},\t accuracy={a}".format(d=degree, l=lambda_, a=results[i_degree, i_lambda]))
            except np.linalg.LinAlgError : 
                results[i_degree, i_lambda] = 0
                print("degree={d},\t lambda={l},\t accuracy={a}".format(d=degree, l=lambda_, a="0 - singular matrix"))

    i,j = np.unravel_index(np.argmax(results, axis=None), results.shape)

    return degrees[i], lambdas[j], results[i, j]

In [ ]:
find_best_least_squares_model(y_jet0, tx_0_filtered)

In [ ]:
def find_best_model(y, tx) : 
    degree_logistic, lambda_logistic, acc_logistic = find_best_logistic_regression_model(y, tx, max_iter=10, gamma=0.01, k_fold=4)
    print("Logistic regression: (degree: {d}, lambda: {l}, accuracy: {a})".format(d=degree_logistic, l=lambda_logistic, a=acc_logistic))
    degree_ls, lambda_ls, acc_ls = find_best_least_squares_model(y, tx, k_fold=4)
    print("Least squares: (degree: {d}, lambda: {l}, accuracy: {a})".format(d=degree_ls, l=lambda_ls, a=acc_ls))

    if acc_logistic > acc_ls : 
        return "logistic regression", degree_logistic, lambda_logistic, acc_logistic
    else : 
        return "least squares", degree_ls, lambda_ls, acc_ls

In [ ]:
def find_model_for_higgs_dataset(y,tx) : 
    
    jet_0_model, jet_0_degree, jet_0_lambda, jet_0_accuracy = find_best_model(y_jet0, tx_0_filtered)
    print("Jet0: model={m}, degree={d}, lambda={l}, accuracy={a}".format(m=jet_0_model,d=jet_0_degree,l=jet_0_lambda,a=jet_0_accuracy))
    jet_1_model, jet_1_degree, jet_1_lambda, jet_1_accuracy = find_best_model(y_jet1, tx_1_filtered)
    print("Jet1: model={m}, degree={d}, lambda={l}, accuracy={a}".format(m=jet_1_model,d=jet_1_degree,l=jet_1_lambda,a=jet_1_accuracy))
    jet_2_model, jet_2_degree, jet_2_lambda, jet_2_accuracy = find_best_model(y_jet2, tx_2_filtered)
    print("Jet2: model={m}, degree={d}, lambda={l}, accuracy={a}".format(m=jet_2_model,d=jet_2_degree,l=jet_2_lambda,a=jet_2_accuracy))
